# Create list of files from MAST download

Credit Jo Taylor.

In [4]:
from astropy.io import fits
import pandas as pd
import glob
import os

## Get list of files with header keywords

In [2]:
# set up an empty dataframe initially to fill in
df = pd.DataFrame(columns=["FILENAME",
                           "TARG_RA", 
                           "TARG_DEC", 
                           "FILTER", # Grism; GR150R/GR150C
                           "PUPIL", # Filter used; F090W, F115W, F140M, F150W F158M, F200W
                           "PATT_NUM", # Position number within dither pattern for WFSS
                           "NUMDTHPT", # Total number of points in entire dither pattern
                           "XOFFSET", # X offset from pattern starting position for NIRISS (arcsec)
                           "YOFFSET"]) # Y offset from pattern starting position for NIRISS (arcsec)

for ratefile in glob.glob("data/*rate.fits"):
    image = fits.open(ratefile)
    
    # create a new dataframe for each file
    df2 = pd.DataFrame({"FILENAME"   : [image[0].header["FILENAME"]],
                        "TARG_RA"     : [image[0].header["TARG_RA"]],
                        "TARG_DEC"     : [image[0].header["TARG_DEC"]],
                        "FILTER"     : [image[0].header["FILTER"]],
                        "PUPIL"      : [image[0].header["PUPIL"]],
                        "PATT_NUM"   : [image[0].header["PATT_NUM"]],
                        "NUMDTHPT"   : [image[0].header["NUMDTHPT"]],
                        "XOFFSET"    : [image[0].header["XOFFSET"]],
                        "YOFFSET"    : [image[0].header["YOFFSET"]]
                       })
    
    # merge the two dataframes together to create a dataframe with all 
    df = pd.concat([df, df2], ignore_index = True, axis = 0)
    
dfsort = df.sort_values('FILENAME', ignore_index = True)

## Save to csv file

In [3]:
dfsort.to_csv("./list_ngdeep_rate.csv",sep=',')

## Move files to preferred location

At this point, I move the rate files and the association files to a preferred place on my machine:
- all rate files under NGDEEP/rate
- level 2 association files under NGDEEP/asn_level2
- level 3 association files under NGDEEP/asn_level3

In [9]:
# first, make all of the new directories
topdir = 'data/NGDEEP'
if not os.path.exists(topdir):
    os.mkdir(topdir)
    print('Created:', topdir)

for subdir in ['rate', 'asn_level2', 'asn_level3']:
    new_subdir = os.path.join(topdir, subdir)
    if not os.path.exists(new_subdir):
        os.mkdir(new_subdir)
        print('Created:', new_subdir)

file_dict = {'rate' : 'rate',
             'image3' : 'asn_level3',
             'image2' : 'asn_level2'}

# now move all of the files to the appropriate locations
for filename in glob.glob('data/*.fits') + glob.glob('data/*.json'):
    try:
        index = filename.split('_')[2] # json files; looking for image2/image3
        subdir = file_dict[index]
    except KeyError:
        try:
            index2 = filename.split('_')[-1].split('.')[0] # rate files
            subdir = file_dict[index2]
        except KeyError:
            print(f'Unrecognized index: {index} or {index2}')
            continue
    
    new_file = os.path.join(topdir, subdir, os.path.basename(filename))
    os.rename(filename, new_file)
    print(f'Moved: {filename} to {new_file}')

Moved: data/jw02079004001_04101_00002_nis_rate.fits to data/NGDEEP/rate/jw02079004001_04101_00002_nis_rate.fits
Moved: data/jw02079004003_06101_00002_nis_rate.fits to data/NGDEEP/rate/jw02079004003_06101_00002_nis_rate.fits
Moved: data/jw02079004002_10101_00001_nis_rate.fits to data/NGDEEP/rate/jw02079004002_10101_00001_nis_rate.fits
Moved: data/jw02079004002_06101_00003_nis_rate.fits to data/NGDEEP/rate/jw02079004002_06101_00003_nis_rate.fits
Moved: data/jw02079004003_02101_00001_nis_rate.fits to data/NGDEEP/rate/jw02079004003_02101_00001_nis_rate.fits
Moved: data/jw02079004002_10101_00002_nis_rate.fits to data/NGDEEP/rate/jw02079004002_10101_00002_nis_rate.fits
Moved: data/jw02079004001_04101_00001_nis_rate.fits to data/NGDEEP/rate/jw02079004001_04101_00001_nis_rate.fits
Moved: data/jw02079004001_12101_00003_nis_rate.fits to data/NGDEEP/rate/jw02079004001_12101_00003_nis_rate.fits
Moved: data/jw02079004003_06101_00001_nis_rate.fits to data/NGDEEP/rate/jw02079004003_06101_00001_nis_ra